In [ ]:
! pip install jsonlines

In [ ]:
import jsonlines
import json

In [ ]:
input_file = './data-chunk-aa.jsonl'
output_file = 'tok_pisin_updated_format.jsonl'

In [ ]:
def get_records(alignment):
    records = []
    
    print(alignment)
    for i in alignment['alignments']:
        eng_start = i["English phrase"]["ranges"][0]["startPosition"]
        mac_start = i["Macula phrase"]["ranges"][0]["startPosition"]
        trg_start = i["Target phrase"]["ranges"][0]["startPosition"]
        eng_end = i["English phrase"]["ranges"][0]["endPosition"]
        mac_end = i["Macula phrase"]["ranges"][0]["endPosition"]
        trg_end = i["Target phrase"]["ranges"][0]["endPosition"]
        
        alignment_position = {
            "references": 
            [
                [{"range": {"startPosition": eng_start, "endPosition": eng_end}}],
                [{"range": {"startPosition": mac_start, "endPosition": mac_end}}],
                [{"range": {"startPosition": trg_start, "endPosition": trg_end}}]
                ]
            }
        
        records.append(alignment_position)

    return records
    

In [ ]:
def get_ids(alignment):
    ids = []
    for i in alignment['macula']['token_ids']:
        ids.append({"id": i['id'], "startPosition": i['range']['startPosition'], "endPosition": i['range']['endPosition']})
        
    return ids

In [ ]:
def convert_format(alignment):
    records = get_records(alignment)
    ids = get_ids(alignment)
    
    new_format = {
        "format": "alignment", 
        "version": "[add timestamp from filename or git commit if you have it]", 
        "groups": [
            {
                "type": "translation", 
                "documents": [
                    {"scheme": "ws-token", "docid": f"{alignment['vref']} (English)"}, 
                    {"scheme": "ws-token", "docid": f"{alignment['vref']} (Macula)", "meta": {"ids": ids}}, 
                    {"scheme": "ws-token", "docid": f"{alignment['vref']} (Target)"}
                ], 
                "roles": ["source", "target", "macula"], 
                "records": records
            }
        ]
    }
    
    return new_format

In [ ]:
with jsonlines.open(input_file) as file:
    for line in file:
        try:
            new_format = str(convert_format(line))
            new_format = new_format.replace('\'','"')
            
            with open(output_file, mode='a') as file:
                json.dump(json.loads(new_format), file)
                file.write('\n')
        except:
            pass
        